In [1]:
!pip install langchain openai requests pandas asyncio

  Using cached langchain-0.3.19-py3-none-any.whl.metadata (7.9 kB)
  Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached langchain_text_splitters-0.3.6-py3-none-any.whl.metadata (1.9 kB)
  Using cached SQLAlchemy-2.0.38-cp312-cp312-win_amd64.whl.metadata (9.9 kB)
  Using cached aiohttp-3.11.13-cp312-cp312-win_amd64.whl.metadata (8.0 kB)
  Using cached numpy-2.2.3-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached pytz-2025.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached aiohappyeyeballs-2.4.6-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.3.2-py2.py3-none-any.whl.metadata (3.8 kB)
  Using cached attrs-25.1.0-py3-none-any.whl.metadata (10 kB)
  Using cached frozenlist-1.5.0-cp312-cp312-win_amd64.whl.metadata (14 kB)
  Using cached multidict-6.1.0-cp312-cp312-win_amd64.whl.metadata (5.1 kB)
  Using cached propcache-0.3.0-cp312-cp312-win_amd64.whl.metadata (10 k


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install langchain-community langchain-core

  Using cached langchain_community-0.3.18-py3-none-any.whl.metadata (2.4 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached httpx_sse-0.4.0-py3-none-any.whl.metadata (9.0 kB)
  Using cached marshmallow-3.26.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached mypy_extensions-1.0.0-py3-none-any.whl.metadata (1.1 kB)
Using cached langchain_community-0.3.18-py3-none-any.whl (2.5 MB)
Using cached dataclasses_json-0.6.7-py3-none-any.whl (28 kB)
Using cached httpx_sse-0.4.0-py3-none-any.whl (7.8 kB)
Using cached marshmallow-3.26.1-py3-none-any.whl (50 kB)
Using cached typing_inspect-0.9.0-py3-none-any.whl (8.8 kB)
Using cached mypy_extensions-1.0.0-py3-none-any.whl (4.7 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import openai
import os
import pandas as pd
# from langchain.prompts import PromptTemplate
# from langchain.llms import OpenAI
# from langchain.chains import LLMChain
# from langchain.agents import initialize_agent, Tool, AgentType
import asyncio
import requests


In [25]:
from langchain_groq import ChatGroq
from langchain.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema import SystemMessage, HumanMessage
from langchain.tools import Tool
from langgraph.graph import StateGraph
from langsmith import trace as langsmith_trace

In [26]:
from dotenv import load_dotenv
load_dotenv() #load all the environment variables

os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

# ## Langsmith Tracking
# os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
# os.environ["LANGCHAIN_TRACING_V2"]="true"
# os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [35]:
## Generative AI: Content Generation with LangChain:
##This section will generate personalized meal plans, exercise routines, and motivational content based on the user’s profile.
# Create the LLM model for text generation

llm=ChatGroq(model_name="gemma2-9b-it", temperature=0)
#llm = OpenAI(temperature=0.7)

# Define Prompt Templates for Meal Plans, Exercise Routines, and Motivation
meal_prompt = """
Generate a personalized 7-day meal plan for a {age}-year-old {dietary_preference} person. They have the following medical history: {medical_history}.
Include breakfast, lunch, dinner, and snack suggestions for each day.
"""

exercise_prompt = """
Create a personalized exercise routine for a {age}-year-old person with a {fitness_level} fitness level. Their medical history includes {medical_history}.
Suggest daily exercises that are gentle on the joints (if necessary) or more intense based on the fitness level.
"""

motivation_prompt = """
Generate motivational content to encourage someone with the following medical condition: {medical_history}.
The user is aiming to improve their fitness and diet. Provide motivational tips and positive affirmations.
"""



# Define Prompt Templates for Meal Plans, Exercise Routines, and Motivation
meal_prompt =ChatPromptTemplate.from_messages(
    [
        ("system","You are an expert professional, health & nutrition consultant and fitness Advicsor. Provide recommendations based on the asked question. Don't say I'm not madical professional"),
        ("user",  """
Generate a personalized 7-day meal plan for a {age}-year-old {dietary_preference} person. They have the following medical history: {medical_history}.
Include breakfast, lunch, dinner, and snack suggestions for each day.
""")
    ]
)

exercise_prompt =ChatPromptTemplate.from_messages(
    [
        ("system","You are an expert professional, health & nutrition consultant and fitness Advicsor. Provide recommendations based on the asked question. Don't say I'm not madical professional"),
        ("user",  """
Create a personalized exercise routine for a {age}-year-old person with a {fitness_level} fitness level. Their medical history includes {medical_history}.
Suggest daily exercises that are gentle on the joints (if necessary) or more intense based on the fitness level.
""")
    ]
)

motivation_prompt =ChatPromptTemplate.from_messages(
    [
        ("system","You are an expert professional, health & nutrition consultant and fitness Advicsor. Provide recommendations based on the asked question. Don't say I'm not madical professional"),
        ("user",  """
Generate motivational content to encourage someone with the following medical condition: {medical_history}.
The user is aiming to improve their fitness and diet. Provide motivational tips and positive affirmations.
""")
    ]
)



# Generate Meal Plan using LangChain
def generate_meal_plan(age, dietary_preference, medical_history):

    meal_chain=meal_prompt|llm
    response=meal_chain.invoke({"age":age, "dietary_preference":dietary_preference, "medical_history":medical_history})
    return response.content
  




# Generate Exercise Routine using LangChain
def generate_exercise_routine(age, fitness_level, medical_history):
    excersise_chain=exercise_prompt|llm
    response=excersise_chain.invoke({"age":age, "fitness_level":fitness_level, "medical_history":medical_history})
    return response.content
  

# Generate Motivational Content using LangChain
def generate_motivation(medical_history):
    medication_chain=motivation_prompt|llm
    response=medication_chain.invoke({"medical_history":medical_history})
    return response.content


In [33]:
response.content

"It's great that you're looking to make healthy choices! \n\nRemember, I'm an AI and this is not a substitute for personalized medical advice. It's crucial to consult with a doctor or registered dietitian to create a meal plan tailored to your specific needs and medical history, especially considering your hypertension. \n\nThat being said, here's a sample 7-day vegetarian meal plan focusing on heart-healthy ingredients that may be beneficial for someone with hypertension:\n\n**General Guidelines:**\n\n* **Focus on fruits, vegetables, and whole grains:** These are naturally low in sodium and rich in potassium, fiber, and antioxidants, all beneficial for blood pressure.\n* **Choose lean protein sources:** Beans, lentils, tofu, tempeh, nuts, and seeds are excellent options.\n* **Limit saturated and trans fats:** Opt for healthy fats like those found in avocados, olive oil, and nuts.\n* **Reduce sodium intake:** Read food labels carefully and choose low-sodium options.\n* **Stay hydrated:

In [39]:
## Example Usage of the Generative AI Functions:

age = 30
dietary_preference = 'vegetarian'
medical_history = 'hypertension'

# Generate the recommendations
meal_plan = generate_meal_plan(age, dietary_preference, medical_history)
exercise_routine = generate_exercise_routine(age, 'beginner', medical_history)
motivation_content = generate_motivation(medical_history)

# print(f"Meal Plan:\n{meal_plan}\n")
# print(f"Exercise Routine:\n{exercise_routine}\n")
print(f"Motivational Content:\n{motivation_content}\n")


Motivational Content:
It's fantastic that you're taking charge of your health and aiming to improve your fitness and diet!  Managing hypertension is a journey, and every step you take towards a healthier lifestyle is a victory. 

Here are some motivational tips and positive affirmations to keep you going:

**Fitness:**

* **Start small and celebrate progress:** Don't feel pressured to jump into intense workouts right away. Begin with 15-20 minutes of walking a few times a week and gradually increase the duration and intensity.  Every step you take is a step in the right direction!
* **Find activities you enjoy:**  Exercise shouldn't feel like a chore. Explore different options like swimming, dancing, cycling, or hiking until you find something that brings you joy. 
* **Make it social:**  Join a fitness class, find a workout buddy, or participate in group activities. Having support and encouragement can make a big difference.
* **Listen to your body:**  Rest when you need to and don't p

In [ ]:
##Agentic AI: Real-Time Decision Making with LangChain Agent:
##This part of the code will handle real-time decision-making for modifying the recommendations based on user progress and feedback.

# Define a tool for handling real-time decision-making (Agentic AI)
def get_progress(user_id):
    # Simulate fetching user progress from a database (e.g., number of workouts completed, current health metrics)
    user_data = {
        'workouts_completed': 10,
        'calories_burned': 3500,
        'diet_score': 8,
    }
    return user_data

def adjust_plan(user_id):
    progress = get_progress(user_id)
    if progress['workouts_completed'] < 5:
        return "It looks like you're just starting! Here's a beginner-friendly exercise routine."
    elif progress['diet_score'] < 6:
        return "Let's work on improving your diet. Here's a more balanced meal plan."
    else:
        return "You're doing great! Keep going with the current plan."

# Integrating LangChain's Agent for continuous decision-making
tools = [
    Tool(
        name="Real-Time Progress Adjustment",
        func=adjust_plan,
        description="Adjust the health plan based on user progress"
    ),
]

agent = initialize_agent(tools, llm, agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Example of how the agent would decide on the next steps
user_id = "user_12345"
response = agent.run(f"Adjust the plan based on progress for user: {user_id}")
print(response)


In [ ]:
#YouTube API Integration for Video Recommendations:
#To recommend YouTube videos based on the user's preferences (e.g., exercise tutorials), you can integrate the YouTube Data API.

def get_youtube_videos(query):
    api_key = 'your_youtube_api_key'
    url = f"https://www.googleapis.com/youtube/v3/search?part=snippet&q={query}&key={api_key}"
    response = requests.get(url)
    video_data = response.json()

    videos = []
    for item in video_data['items']:
        title = item['snippet']['title']
        url = f"https://www.youtube.com/watch?v={item['id']['videoId']}"
        videos.append({"title": title, "url": url})

    return videos

# Example usage: Fetch exercise videos
exercise_videos = get_youtube_videos("beginner exercise routine for seniors")
for video in exercise_videos:
    print(f"Title: {video['title']}, URL: {video['url']}")


In [ ]:
##Bringing It All Together:
#You can combine these pieces to create a full-fledged agent that tailors the health and lifestyle plan based on user data, generates content, adjusts the plan, and even suggests YouTube videos.

#Final Example: Full User Interaction:

In [ ]:
# User Input (normally collected from a form or app interface)
user_profile = {
    'age': 30,
    'dietary_preference': 'vegetarian',
    'fitness_level': 'beginner',
    'medical_history': 'hypertension',
    'user_id': 'user_12345'
}

# Generate Personalized Recommendations
meal_plan = generate_meal_plan(user_profile['age'], user_profile['dietary_preference'], user_profile['medical_history'])
exercise_routine = generate_exercise_routine(user_profile['age'], user_profile['fitness_level'], user_profile['medical_history'])
motivation_content = generate_motivation(user_profile['medical_history'])

# Decision Making via Agentic AI
response = agent.run(f"Adjust the plan based on progress for user: {user_profile['user_id']}")

# YouTube Video Recommendations
exercise_videos = get_youtube_videos("beginner exercise for hypertension")

# Display Recommendations
print(f"Meal Plan:\n{meal_plan}\n")
print(f"Exercise Routine:\n{exercise_routine}\n")
print(f"Motivational Content:\n{motivation_content}\n")
print(f"Adjustments Based on Progress:\n{response}\n")
print("Recommended YouTube Videos:")
for video in exercise_videos:
    print(f"Title: {video['title']}, URL: {video['url']}")
